# Data Collection Step 2

The code below imports our data collected during step 1 and will take from that information the listing_url to parse listing page by listing page and retrieve all further details therein. 

**We are grabbing the following values for each listing:**

| Value Name | Description |
|:---|:---|
| shop_name | name of the Etsy shop, could be used to create a URL for those shop's listings for further scraping |
| start_price | minimum price for this listing item, prices may scale with size or extra wishes |
| size_options | size options that are offered for this dress |
| ship_cost_germany |  |
| allows_returns |  |
| shop_location |  |
| shop_5_star_rating_percentage |  |
| shop_4_star_rating_percentage |  |
| shop_3_star_rating_percentage |  |
| shop_2_star_rating_percentage |  |
| shop_1_star_rating_percentage |  |
| shop_reviews_count |  |
| item_reviews_count |  |
| listing_customer_reviews |  |
| total_shop_sales |  |

#### Importing required libraries

In [1]:
import pandas as pd
import requests
requests.adapters.DEFAULT_RETRIES = 5 # increase retries number
from bs4 import BeautifulSoup
from random import randint
from time import sleep
import tqdm
import re

#### Step2 Scraper Function:

In [2]:
def step2(df_link, default=0):
    
    
    # define empty value lists:
    cols = ['listing_url',
            'shop_name',
            'size_options',            
            'shop_location',
            'allows_returns',
            'total_shop_sales',
            'shop_reviews_count',
            'shop_5_star_rating_percentage',
            'shop_4_star_rating_percentage',
            'shop_3_star_rating_percentage',
            'shop_2_star_rating_percentage',
            'shop_1_star_rating_percentage',
            'item_reviews_count',
            'listing_customer_reviews',
            'listing_customer_reviews',
            'listing_customer_reviews',
            'listing_customer_reviews',
            'start_price',
            'ship_cost_germany',]
    
    errors = []
    
    df = pd.DataFrame(columns = cols)
  
    # turn listing_url into link list for interative loop:
    link = list(df_link["listing_url"])
    
    # define a regular expression pattern trained on DE currentcy format with decimal "," separator
    currency = re.compile('[-+]? (?: (?: \d* \, \d+ ) | (?: \d+ \,? ) )(?: [Ee] [+-]? \d+ ) ?', re.VERBOSE)
    
    #define a regular expression pattern for finding total sales
    sales_select = '[\d]{1},[\d]{3} sales|[\d]{2},[\d]{3} sales|[\d]{3} sales|[\d]{2} sales|[\d]{1} sales'

   
    # english url soup
    for url in link: # tqdm code for progress bar
        response = requests.get(url, timeout=3)
        soup = BeautifulSoup(response.text,"html.parser")
        if response.status_code == 200:  
            row = []
            row.append(url)
            

            # extract shop name, set up failsave in case listing has been removed
            if True:
                try:
                    name = soup.select("#listing-page-cart > div:nth-child(1) > div > div.wt-display-flex-xs.wt-align-items-center.wt-mb-xs-1 > p")[0].get_text()
                    name = name[31:]
                    name = name[:-27]
                    row.append(name)
                except:
                    row.append('No name')


                # extract size options count for this item
                try:
                    sizes = len(soup.select("#listing-page-cart > div.wt-mb-xs-6.wt-mb-lg-0 > div:nth-child(1) > div.wt-mb-xs-3 > div:nth-child(4) > div")[0].select("option")[1:])
                    row.append(sizes)
                except:
                    row.append(0)

                try:
                    # extract shop location
                    location = soup.select("#shipping-variant-div > div > div.wt-grid.wt-mb-xs-3 > div.wt-grid__item-xs-12.wt-text-black.wt-text-caption")[0].get_text()
                    location = location[16:]
                    location = location[:-1]
                    row.append(location)
                except:
                    row.append("N/A")

                
                try:
                    # extract returns allowed yes/no
                    ret = soup.select("#shipping-variant-div > div > div.wt-grid.wt-mb-xs-3")[0].get_text()
                    if ret.find("Accepted") != -1:
                        ret = 1
                    else:
                        ret = 0
                    row.append(ret)
                except:
                    row.append("N/A")


                # extract total shop sales
                try:
                    tot_sales = re.findall(sales_select, soup.select("#listing-page-cart")[0].text)[0]
                    tot_sales = int(tot_sales[:-6].replace(",",""))
                    row.append(tot_sales)
                except:
                    row.append(0)

                    
                # extract review count for the shop 
                try:
                    shop_rev = int(soup.select("#listing-right-column > div > div.body-wrap.wt-body-max-width.wt-display-flex-md.wt-flex-direction-column-xs > div.listing-info.review-col.wt-order-xs-6 > div > div > div:nth-child(2)")[0].get_text()[19:].split(" ")[0].replace(",",""))
                    row.append(shop_rev)
                except:
                    row.append(0)

                
                #extract 5/5 star rating for this shop (only way to do this is in percentage counts per star rating level)
                try:
                    five = soup.select("#listing-right-column > div > div.body-wrap.wt-body-max-width.wt-display-flex-md.wt-flex-direction-column-xs > div.listing-info.review-col.wt-order-xs-6 > div > div > div:nth-child(2)")[0].text.split("5 out of 5 stars\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n5 stars\n\n\n\n\n\n\n")[1]
                    five = float(five.split("%\n\n\n\n\n4 stars")[0])
                    five = five/100
                    row.append(five)
                except:
                    row.append(0)
                    
                try:
                    four = soup.select("#listing-right-column > div > div.body-wrap.wt-body-max-width.wt-display-flex-md.wt-flex-direction-column-xs > div.listing-info.review-col.wt-order-xs-6 > div > div > div:nth-child(2)")[0].text.split("4 stars\n\n\n\n\n\n\n")[1]
                    four = float(four.split("%\n\n\n\n\n3 stars")[0])
                    four = four/100
                    row.append(four)
                except:
                    row.append(0)
                    
                try:
                    three = soup.select("#listing-right-column > div > div.body-wrap.wt-body-max-width.wt-display-flex-md.wt-flex-direction-column-xs > div.listing-info.review-col.wt-order-xs-6 > div > div > div:nth-child(2)")[0].text.split("3 stars\n\n\n\n\n\n\n")[1]
                    three = float(three.split("%\n\n\n\n\n")[0])
                    three = three/100
                    row.append(three)
                except:
                    row.append(0)
                    
                try:
                    two = soup.select("#listing-right-column > div > div.body-wrap.wt-body-max-width.wt-display-flex-md.wt-flex-direction-column-xs > div.listing-info.review-col.wt-order-xs-6 > div > div > div:nth-child(2)")[0].text.split("2 stars\n\n\n\n\n\n\n")[1]
                    two = float(two.split("%\n\n\n\n\n")[0])
                    two = two/100
                    row.append(two)
                except:
                    row.append(0)
                
                try:
                    one = soup.select("#listing-right-column > div > div.body-wrap.wt-body-max-width.wt-display-flex-md.wt-flex-direction-column-xs > div.listing-info.review-col.wt-order-xs-6 > div > div > div:nth-child(2)")[0].text.split("1 star\n\n\n\n\n\n\n")[1]
                    one = float(one.split("%\n\n\n\n\n")[0])
                    one = one/100
                    row.append(one)
                except:
                    row.append(0)
                
                
                # extract review count for this item
                try:
                    item_rev = soup.select("#listing-right-column > div > div.body-wrap.wt-body-max-width.wt-display-flex-md.wt-flex-direction-column-xs > div.listing-info.review-col.wt-order-xs-6 > div > div > div:nth-child(2)")[0].text.split("Reviews for this item\n                \n                    ")[1]
                    item_rev = int(item_rev.split("\n")[0].replace(",",""))
                    row.append(item_rev)
                except:
                    row.append(0)
                
                
                # extract customer reviews per item
                try:
                    if soup.select("#listing-right-column > div > div.body-wrap.wt-body-max-width.wt-display-flex-md.wt-flex-direction-column-xs > div.listing-info.review-col.wt-order-xs-6 > div > div > div:nth-child(2)")[0].text.find("Reviews for this item") != -1:
                        rev_key = "#review-preview-toggle-{}"
                        for iterate in range(0,4):
                            key = rev_key.format(iterate)
                            try:
                                rev = soup.select(key)[0].text.split("\n                    ")[1]
                                rev = rev.split(" \n\n    ")[0]
                                row.append(rev)
                            except:
                                row.append("N.A.")
                    else:
                        row.extend(["N.A.","N.A.","N.A.","N.A."])
                except:
                    row.extend(["N.A.","N.A.","N.A.","N.A."])

                    
                # create DE url variant to scrape prices in EUR instead of USD
                flag = True
                try:
                    url_de = url.replace("https://www.etsy.com/listing/","https://www.etsy.com/de/listing/")
                    response_de = requests.get(url_de)
                    soup_de = BeautifulSoup(response_de.text,"html.parser")     
                except:
                    flag = False
                    row.append(0)
                    row.append(0)

                if flag == True:
                    
                    # extract starting price quote from soup and applying proper decimal "." separator
                    try:
                        p = currency.findall(soup_de.select("#listing-page-cart > div.wt-mb-xs-6.wt-mb-lg-0 > div:nth-child(1) > div.wt-mb-xs-3 > div.wt-mb-xs-3 > div:nth-child(1) > div > div.wt-display-flex-xs.wt-align-items-center.wt-flex-wrap > p")[0].get_text())[0]
                        p = float(p.replace(",","."))
                        row.append(p)
                    except:
                        row.append(0)

                        
                    # extract shipping to Germany cost quote
                    try:
                        sc = soup_de.select("#shipping-variant-div > div > div.wt-grid.wt-mb-xs-3")[0].text.split('\nVersandkosten\n')[1]
                        sc = sc.split('\n\n\n\n\n\n')[0]
                        if sc == 'Kostenlos':
                            ship_cost = 0           
                        else:
                            ship_cost = float(sc[:-2].replace(",","."))
                        row.append(ship_cost)
                    except:
                        row.append(0)                 

            
            df.loc[len(df)] = row


            # wait time between cycles
            wait_time = randint(1,2500)
            sleep(wait_time/1000)

    return df

#### Preforming step 2 on first step1 dataset in batches

In [42]:
data = pd.read_csv('step1_data.csv')
full_df = pd.DataFrame()
for i in tqdm.tqdm(range(5,len(data),5)):
    df = step2(data[i-5:i])
    full_df = pd.concat([full_df,df])

100%|███████████████████████████████████████| 399/399 [3:33:02<00:00, 32.04s/it]


In [50]:
full_df.shape

(1995, 19)

In [ ]:
data = pd.read_csv('step1_data.csv')

In [54]:
full_df.shape

(1995, 19)

In [47]:
result = pd.merge(data, full_df, on="listing_url")

In [57]:
result.shape

(1995, 21)

In [58]:
result.to_csv('step2_primary.csv',index=False)

In [15]:
data = pd.read_csv('step1_bohip_christening_baptism_beige_formal.csv')
data.shape

(9952, 3)

In [9]:
data=data.tail(4300)

In [10]:
data.shape

(4300, 3)

In [16]:
full_df = pd.DataFrame()

for i in tqdm.tqdm(range(10,len(data),10)):
    df = step2(data[i-10:i])
    full_df = pd.concat([full_df,df], axis=0)

 38%|██████████████                       | 378/995 [5:13:52<8:32:20, 49.82s/it]


ConnectionError: HTTPSConnectionPool(host='www.etsy.com', port=443): Max retries exceeded with url: /listing/1062980080/baby-girl-flower-girl-dress-infant?ga_order=highest_reviews&ga_search_type=all&ga_view_type=gallery&ga_search_query=flower+girl+dress&ref=sr_gallery-224-4&organic_search_click=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='www.etsy.com', port=443): Read timed out. (read timeout=3)"))

In [17]:
full_df.shape

(3780, 19)

In [18]:
full_df.to_csv('step2_2.csv',index=False)

In [70]:
etsy_repo.shape

(5, 19)